# Task 4


In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import os
from google.colab import drive
import numpy as np

TensorFlow 2.x selected.
2.1.0-rc1


In [2]:
import os
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
os.listdir()


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['.config', 'drive', 'sample_data']

In [3]:
base_dir = 'drive/My Drive/Computational Intelligence - MY PROJECT/My_Project_CompInt' 
os.listdir(base_dir)

['OldNotebooks',
 'Task 1',
 'Models',
 'Task4_RelevantPapers',
 'Tensors',
 'Rasla_Davide_505828',
 'Copia di Task 3.1_ INCEPTIONV3.ipynb',
 'Task4.ipynb']

# Getting the Data 

In [0]:
from tensorflow.keras.utils import to_categorical
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images, train_labels, test_images, test_lable = load_training()




## Reshape

In [0]:

train_images = train_images.reshape((5352, 150, 150,1))
train_images = train_images.astype('float32') / 65535




test_images = test_images.reshape((672, 150, 150,1))
test_images = test_images.astype('float32') / 65535


In [0]:
print(train_images[0:100])

## To categorical

In [0]:
#Se uso softmax
from keras.utils import to_categorical
LabelArray_Categorical = to_categorical(train_labels,5)
Test_label_categorical = to_categorical(test_lable,5)

In [0]:
print(LabelArray_Categorical[0:1000])
print(LabelArray_Categorical[2000:])

## Modifiyng the test set

### SPLITTING AND CATEGORICAL

In [0]:
import numpy as np
keras.layers.Conv2D?
keras.layers.MaxPooling2D?
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Test_baseline = []
Test_abnormality = []
Test_baseline_target = []
Test_abnormality_target = []
                      
#splitting
for i in range(0, len(test_images)): 
    if (i % 2)==0: 
        Test_baseline.append(test_images[i])
        Test_baseline_target.append(test_lable[i])
    else : 
        Test_abnormality.append(test_images[i])
        Test_abnormality_target.append(test_lable[i])
        
#editing labels
leng = len(Test_abnormality_target)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target)
for i in range(0, leng):
    if Test_abnormality_target[i] == 1:
       Test_abnormality_target_4[i] = 0
    if Test_abnormality_target[i] == 2:
       Test_abnormality_target_4[i] = 1
    if Test_abnormality_target[i] == 3:
       Test_abnormality_target_4[i] = 2
    if Test_abnormality_target[i] == 4:
       Test_abnormality_target_4[i] = 3

#categorical

Testabnormality_new = np.array(Test_abnormality)
Testbaseline_new = np.array(Test_baseline)

Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target)

In [0]:
print(Test_abnormality_target_Categorical)

# USING SIAMESE NETWORK FROM SCRATCH

Based on: https://keras.io/examples/mnist_siamese/


Define the two siamese net


In [0]:
from tensorflow.keras import layers
def get_siamese_model(input_shape):


    # Define the tensors for the two input images
    left_input = keras.layers.Input(input_shape)
    right_input = keras.layers.Input(input_shape)
    
    # Convolutional Neural Network
    model = keras.models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation="relu",input_shape=(150, 150, 1),padding='same',strides=1))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu",padding='same',strides=1,kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (2, 2), activation="relu",padding='same',strides=1,kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (2, 2), activation="relu",padding='same',strides=1,kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(layers.Flatten())
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    Subtracted = keras.layers.Subtract()([encoded_l,encoded_r])
    dense1 = keras.layers.Dense(256, activation="relu")(Subtracted)
    dropout2 = keras.layers.Dropout(0.4)(dense1)
    dense2 = keras.layers.Dense(128, activation="relu")(dropout2)
    out = keras.layers.Dense(4, activation='softmax')(dense2)


    # Connect the inputs with the outputs
    siamese_net = keras.models.Model(inputs=[left_input,right_input],outputs=out)
    
    # return the model
    return siamese_net

input_shape = (150,150,1)

model  = get_siamese_model (input_shape)


In [0]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

## Data Manipulation

In [0]:
from tensorflow.keras.utils import to_categorical
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))
test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))
train_images_Augmented = train_images_Augmented.astype('float32') / 65535
test_images_Augmented = test_images_Augmented.astype('float32') / 65535

In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 2
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 3


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,4)


In [0]:
#DATA
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 2
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 3

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,4)
baseline_target_Categorical = to_categorical(baseline_target,4)



print(abnormality_target_Categorical[:10])

In [0]:
print(abnormality_Not_Shuffled[3])

## Cross_validation

In [0]:
#cross-validation, SUL TRAINING ESTRATTO e shuffled
import numpy as np
keras.layers.Conv2D?
keras.layers.MaxPooling2D?
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def plot():
  acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()
  return


############################# CROSS VALIDATION ######################

k = 10

num_epochs = 40
History_Arr =[]
input_shape = (150,150,1)
Cross_model = get_siamese_model(input_shape)
Cross_model.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['categorical_accuracy'])
History_Arr =[]

print(len(abnormality))
print(len(baseline))
print(abnormality_target_Categorical)


num_val_samples = len(baseline) // k
for i in range(k):
  print('processing fold #', i)
  val_data_ABNORMALITY =  abnormality[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets_ABNORMALITY = abnormality_target_Categorical[i * num_val_samples: (i + 1) * num_val_samples]
  val_data_BASELINE = baseline[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data_ABN =    np.concatenate([abnormality[:i * num_val_samples],abnormality[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets_ABN = np.concatenate([abnormality_target_Categorical[:i * num_val_samples],abnormality_target_Categorical[(i + 1) * num_val_samples:]],axis=0)
  partial_train_data_BASE = np.concatenate([baseline[:i * num_val_samples],baseline[(i + 1) * num_val_samples:]],axis=0)
  history = Cross_model.fit([partial_train_data_BASE,partial_train_data_ABN ], partial_train_targets_ABN, validation_data=((val_data_BASELINE,val_data_ABNORMALITY), val_targets_ABNORMALITY), epochs=num_epochs, batch_size=32)
  History_Arr.append(history)
  Cross_model = get_siamese_model(input_shape)
  Cross_model.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  plot()


In [0]:
print(abnormality_new)

## Fitting

In [0]:
 history = model.fit([baseline,abnormality ], abnormality_target_Categorical, epochs=30
                             , batch_size=32)

## Evalutate the model

In [0]:
#evaluate the model

test_loss, test_acc = model.evaluate((Test_baseline_Not_Shuffled, Test_abnormality_Not_Shuffled), Test_abnormality_target_Categorical, verbose= 1)
print(test_loss)
print(test_acc)

## Adding Data Augmentation 

### Data Augmentation

In [0]:
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))
test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))
test_images_Augmented = test_images_Augmented.astype('float32') / 65535


Test manipulation

In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 2
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 3


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,4)


In [0]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 2
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 3

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,4)
baseline_target_Categorical = to_categorical(baseline_target,4)



print(abnormality_target_Categorical[:10])

In [0]:
print(Test_abnormality_Not_Shuffled)

In [0]:
#ABNORMALITY OPERATIONS

print(abnormality_target)

In [0]:
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def double_generator(train_base_images, train_abn_images, train_labels, subset, batch_size=32):

    gen = ImageDataGenerator(
      validation_split=0.1,
      rescale=1./65535,
      rotation_range=360,
      zoom_range=0.1, 
      horizontal_flip=True,
      vertical_flip = True,
      fill_mode='nearest'
    )

    gen.fit(train_abn_images)

    gen_abn = gen.flow(train_abn_images, train_labels,  batch_size=batch_size, subset=subset, seed=1)
    gen_base = gen.flow(train_base_images, train_labels, batch_size=batch_size, subset=subset, seed=1)

    while True:
        abn_img, abn_label = gen_abn.next()
        base_img, _ = gen_base.next()
        yield [base_img, abn_img], abn_label

train_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'training', batch_size=32)
validation_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'validation', batch_size=32)

### FITTING

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2408//32,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=268//32)

### Evalutate the model

In [0]:


test_loss, test_acc = model.evaluate((Test_baseline_Not_Shuffled, Test_abnormality_Not_Shuffled),Test_abnormality_target_Categorical,batch_size=32,  verbose= 1)
print(test_loss)
print(test_acc)

### Load OR Save the model

In [0]:
#Save the model!
model.save(os.path.join(base_dir,'Models/Model_Task_4_Siamese_WithDataAugmentation_0.52.h5'))

# USING DCCNN (Deep - Columnar)

## Model 

In [13]:
%tensorflow_version 1.14
!pip install keras==2.1.5
import tensorflow as tf
from tensorflow import keras
from keras.layers import merge, Input
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D

img_rows, img_cols = 150, 150

nb_filters_1 = 32
nb_filters_2 = 64
nb_filters_3 = 128
nb_filters_4 = 192
nb_conv = 3

init1 = keras.layers.Input(shape=(150,150,1),) #channel 1


init2 = keras.layers.Input(shape=(150,150,1),) #channel 2

fork11 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv), strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init1)
fork12 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init2)
merge1 = keras.layers.Subtract()([fork11, fork12,])
maxpool1 = keras.layers.MaxPooling2D(strides=(2,2),)(merge1)

fork21 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool1)
fork22 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool1)
merge2 = keras.layers.Subtract()([fork21, fork22, ])
maxpool2 = keras.layers.MaxPooling2D(strides=(2,2), )(merge2)

fork31 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
fork32 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
merge3 = keras.layers.Subtract()([fork31, fork32, ]) 
maxpool3 = keras.layers.MaxPooling2D(strides=(2,2), )(merge3)

fork41 = keras.layers.Conv2D(nb_filters_4, (3, 3),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool3)
fork42 = keras.layers.Conv2D(nb_filters_4, (3, 3), strides=(1,1), activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool3)
merge4 = keras.layers.Subtract()([fork41, fork42, ]) 
maxpool4 = keras.layers.MaxPooling2D(strides=(2,2), )(merge4)



flatten = keras.layers.Flatten()(maxpool4)
dense1 = keras.layers.Dense(192, activation="relu")(flatten)
dropout = keras.layers.Dropout(0.4)(dense1)
dense2 = keras.layers.Dense(128, activation="relu")(dropout)
dense3 = keras.layers.Dense(64, activation="relu")(dense2)
output = keras.layers.Dense(4, activation="softmax")(dense3)


DCCNN_MODEL = keras.models.Model((init1, init2), output)


`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
     |████████████████████████████████| 337kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
DCCNN_MODEL.summary()

In [0]:
DCCNN_MODEL.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

## Data Manipulation

In [0]:
from tensorflow.keras.utils import to_categorical
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))


test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))


train_images_Augmented = train_images_Augmented.astype('float32') / 65535


test_images_Augmented = test_images_Augmented.astype('float32') / 65535

In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 2
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 3


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,4)


In [0]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 2
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 3

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,4)
baseline_target_Categorical = to_categorical(baseline_target,4)



print(abnormality_target_Categorical[:10])

In [0]:

print(abnormality_target)

## 10-Cross validation

In [0]:
#cross-validation, SUL TRAINING ESTRATTO e shuffled
import numpy as np
keras.layers.Conv2D?
keras.layers.MaxPooling2D?
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def get_model():

  nb_filters_1 = 32
  nb_filters_2 = 64
  nb_filters_3 = 128
  nb_filters_4 = 192
  nb_conv = 3

  init1 = keras.layers.Input(shape=(150,150,1),) #channel 1


  init2 = keras.layers.Input(shape=(150,150,1),) #channel 2

  fork11 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv), strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init1)
  fork12 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init2)
  merge1 = keras.layers.Subtract()([fork11, fork12,])
  maxpool1 = keras.layers.MaxPooling2D(strides=(2,2),)(merge1)

  fork21 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool1)
  fork22 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool1)
  merge2 = keras.layers.Subtract()([fork21, fork22, ])
  maxpool2 = keras.layers.MaxPooling2D(strides=(2,2), )(merge2)

  fork31 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
  fork32 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
  merge3 = keras.layers.Subtract()([fork31, fork32, ]) 
  maxpool3 = keras.layers.MaxPooling2D(strides=(2,2), )(merge3)

  fork41 = keras.layers.Conv2D(nb_filters_4, (3, 3),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool3)
  fork42 = keras.layers.Conv2D(nb_filters_4, (3, 3), strides=(1,1), activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool3)
  merge4 = keras.layers.Subtract()([fork41, fork42, ]) 
  maxpool4 = keras.layers.MaxPooling2D(strides=(2,2), )(merge4)



  flatten = keras.layers.Flatten()(maxpool4)
  dense1 = keras.layers.Dense(192, activation="relu")(flatten)
  dropout = keras.layers.Dropout(0.5)(dense1)
  dense2 = keras.layers.Dense(128, activation="relu")(dropout)
  dense3 = keras.layers.Dense(64, activation="relu")(dense2)
  output = keras.layers.Dense(4, activation="softmax")(dense3)
  output = keras.layers.Dense(4, activation="softmax")(dense3)

  Cross_model = keras.models.Model((init1, init2), output)
  return Cross_model

def plot():
  acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()
  return


############################# CROSS VALIDATION ######################

k = 10

num_epochs = 40
History_Arr =[]
Cross_model = get_model()
Cross_model.compile(optimizer='RMSprop',loss='categorical_crossentropy', metrics=['categorical_accuracy'])


num_val_samples = len(baseline) // k
for i in range(k):
  print('processing fold #', i)
  val_data_ABNORMALITY =  abnormality[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets_ABNORMALITY = abnormality_target_Categorical[i * num_val_samples: (i + 1) * num_val_samples]
  val_data_BASELINE = baseline[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data_ABN =    np.concatenate([abnormality[:i * num_val_samples],abnormality[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets_ABN = np.concatenate([abnormality_target_Categorical[:i * num_val_samples],abnormality_target_Categorical[(i + 1) * num_val_samples:]],axis=0)
  partial_train_data_BASE = np.concatenate([baseline[:i * num_val_samples],baseline[(i + 1) * num_val_samples:]],axis=0)
  history = Cross_model.fit([partial_train_data_BASE,partial_train_data_ABN ], partial_train_targets_ABN, validation_data=((val_data_BASELINE,val_data_ABNORMALITY), val_targets_ABNORMALITY), epochs=num_epochs, batch_size=32)
  History_Arr.append(history)
  Cross_model = get_model()
  Cross_model.compile(optimizer='RMSprop',loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  plot()

## FITTING COMPLETO
Dopo la cross validation, fitto su tutto il DATASET

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

num_epochs = 40
history = DCCNN_MODEL.fit([baseline,abnormality ], abnormality_target_Categorical ,   epochs=num_epochs, batch_size=32)

## Evalutate the model

In [0]:
#evaluate the model

test_loss, test_acc = DCCNN_MODEL.evaluate((Test_baseline_Not_Shuffled,Test_abnormality_Not_Shuffled), Test_abnormality_target_Categorical,batch_size=32, verbose= 1)
print(test_loss)
print(test_acc)

## Load OR Save the model

In [0]:
model1 = tf.keras.models.load_model(os.path.join(base_dir,'Models/Model_Task_4_DCCN_BaseVersion_0.55.h5'))

## Adding Data Augmentation 

### Data Augmentation

In [0]:
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))
test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))
test_images_Augmented = test_images_Augmented.astype('float32') / 65535


Test manipulation

In [7]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 2
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 3


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,4)


Using TensorFlow backend.


In [0]:
print(Test_abnormality_Not_Shuffled)

In [8]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 2
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 3

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,4)
baseline_target_Categorical = to_categorical(baseline_target,4)



print(abnormality_target_Categorical[:10])

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [0]:
#ABNORMALITY OPERATIONS

print(abnormality_target)

In [0]:
print(base_train_data)
print(base_train_data.shape)

In [0]:
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def double_generator(train_base_images, train_abn_images, train_labels, subset, batch_size=32):

    gen = ImageDataGenerator(
      validation_split=0.1,
      rescale=1./65535,
      rotation_range=360,
      zoom_range=0.2, 
      horizontal_flip=True,
      vertical_flip = True,
      fill_mode='nearest'
    )

    gen.fit(train_abn_images)

    gen_abn = gen.flow(train_abn_images, train_labels,  batch_size=batch_size, subset=subset, seed=1)
    gen_base = gen.flow(train_base_images, train_labels, batch_size=batch_size, subset=subset, seed=1)

    while True:
        abn_img, abn_label = gen_abn.next()
        base_img, _ = gen_base.next()
        yield [base_img, abn_img], abn_label

train_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'training', batch_size=32)
validation_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'validation', batch_size=32)

### FITTING

In [0]:
history = DCCNN_MODEL.fit_generator(
      train_generator,
      steps_per_epoch=2408//32,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=268//32)

### Evalutate the model

In [27]:


test_loss, test_acc = DCCNN_MODEL.evaluate((Test_baseline_Not_Shuffled, Test_abnormality_Not_Shuffled),Test_abnormality_target_Categorical,batch_size=32,  verbose= 1)
print(test_loss)
print(test_acc)

336/336 [==============================] - 0s 431us/sample - loss: 0.8652 - categorical_accuracy: 0.6577
0.8652134906677973
0.6577381


### Load OR Save the model

In [0]:
#Save the model!
DCCNN_MODEL.save(os.path.join(base_dir,'Models/Model_Task_4_DCCNN_Corretto_WithDataAugmentation_0.6578.h5'))

# USING DCCNN with baseline for binary classification (Deep - Columnar)



## Model 

In [0]:
%tensorflow_version 1.14
!pip install keras==2.1.5
import tensorflow as tf
from tensorflow import keras
from keras.layers import merge, Input
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D

img_rows, img_cols = 150, 150

nb_filters_1 = 32
nb_filters_2 = 64
nb_filters_3 = 128
nb_filters_4 = 192
nb_conv = 3

init1 = keras.layers.Input(shape=(150,150,1),) #channel 1


init2 = keras.layers.Input(shape=(150,150,1),) #channel 2

fork11 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv), strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init1)
fork12 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init2)
merge1 = keras.layers.Subtract()([fork11, fork12,])
maxpool1 = keras.layers.MaxPooling2D(strides=(2,2),)(merge1)

fork21 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool1)
fork22 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool1)
merge2 = keras.layers.Subtract()([fork21, fork22, ])
maxpool2 = keras.layers.MaxPooling2D(strides=(2,2), )(merge2)

fork31 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
fork32 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
merge3 = keras.layers.Subtract()([fork31, fork32, ]) 
maxpool3 = keras.layers.MaxPooling2D(strides=(2,2), )(merge3)

fork41 = keras.layers.Conv2D(nb_filters_4, (3, 3),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool3)
fork42 = keras.layers.Conv2D(nb_filters_4, (3, 3), strides=(1,1), activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool3)
merge4 = keras.layers.Subtract()([fork41, fork42, ]) 
maxpool4 = keras.layers.MaxPooling2D(strides=(2,2), )(merge4)



flatten = keras.layers.Flatten()(maxpool4)
dense1 = keras.layers.Dense(192, activation="relu")(flatten)
dropout = keras.layers.Dropout(0.5)(dense1)
dense2 = keras.layers.Dense(128, activation="relu")(dropout)
dense3 = keras.layers.Dense(64, activation="relu")(dense2)
output = keras.layers.Dense(2, activation="softmax")(dense3)


DCCNN_MODEL = keras.models.Model((init1, init2), output)


In [0]:
DCCNN_MODEL.summary()

In [0]:
DCCNN_MODEL.compile(optimizer='Adam',
            loss='binary_crossentropy', metrics=['binary_accuracy'])

## Data Manipulation

In [0]:
from tensorflow.keras.utils import to_categorical
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))
train_images_Augmented = train_images_Augmented.astype('float32') / 65535

test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))
test_images_Augmented = test_images_Augmented.astype('float32') / 65535


In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 1


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,2)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,2)


In [0]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 1

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,2)
baseline_target_Categorical = to_categorical(baseline_target,2)



print(abnormality_target_Categorical[:10])

In [0]:

print(abnormality_target)

## 10-Cross validation

In [0]:
#cross-validation, SUL TRAINING ESTRATTO e shuffled
import numpy as np
keras.layers.Conv2D?
keras.layers.MaxPooling2D?
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def get_model():

  img_rows, img_cols = 150, 150

  nb_filters_1 = 64
  nb_filters_2 = 129
  nb_filters_3 = 256
  nb_filters_4 = 192
  nb_conv = 3

  init1 = keras.layers.Input(shape=(150,150,1),) #channel 1


  init2 = keras.layers.Input(shape=(150,150,1),) #channel 2

  fork11 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv), strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init1)
  fork12 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(init2)
  merge1 = keras.layers.Subtract()([fork11, fork12,])
  maxpool1 = keras.layers.MaxPooling2D(strides=(2,2),)(merge1)

  fork21 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool1)
  fork22 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool1)
  merge2 = keras.layers.Subtract()([fork21, fork22, ])
  maxpool2 = keras.layers.MaxPooling2D(strides=(2,2), )(merge2)

  fork31 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
  fork32 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation="relu",  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool2)
  merge3 = keras.layers.Subtract()([fork31, fork32, ]) 
  maxpool3 = keras.layers.MaxPooling2D(strides=(2,2), )(merge3)

  fork41 = keras.layers.Conv2D(nb_filters_4, (3, 3),strides=(1,1),  activation="relu", kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool3)
  fork42 = keras.layers.Conv2D(nb_filters_4, (3, 3), strides=(1,1), activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(maxpool3)
  merge4 = keras.layers.Subtract()([fork41, fork42, ]) 
  maxpool4 = keras.layers.MaxPooling2D(strides=(2,2), )(merge4)



  flatten = keras.layers.Flatten()(maxpool4)
  dense1 = keras.layers.Dense(192, activation="relu")(flatten)
  dropout = keras.layers.Dropout(0.5)(dense1)
  dense2 = keras.layers.Dense(128, activation="relu")(dropout)
  dense3 = keras.layers.Dense(64, activation="relu")(dense2)
  output = keras.layers.Dense(2, activation="softmax")(dense3)

  Cross_model = keras.models.Model((init1, init2), output)
  return Cross_model

def plot():
  acc = history.history['binary_crossentropy']
  val_acc = history.history['val_binary_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()
  return


############################# CROSS VALIDATION ######################

k = 10

num_epochs = 40
History_Arr =[]
Cross_model = get_model()
Cross_model.compile(optimizer='RMSprop',loss='binary_crossentropy', metrics=['binary_accuracy'])

print(abnormality_target_Categorical)
num_val_samples = len(baseline) // k
for i in range(k):
  print('processing fold #', i)
  val_data_ABNORMALITY =  abnormality[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets_ABNORMALITY = abnormality_target_Categorical[i * num_val_samples: (i + 1) * num_val_samples]
  val_data_BASELINE = baseline[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data_ABN =    np.concatenate([abnormality[:i * num_val_samples],abnormality[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets_ABN = np.concatenate([abnormality_target_Categorical[:i * num_val_samples],abnormality_target_Categorical[(i + 1) * num_val_samples:]],axis=0)
  partial_train_data_BASE = np.concatenate([baseline[:i * num_val_samples],baseline[(i + 1) * num_val_samples:]],axis=0)
  history = Cross_model.fit([partial_train_data_BASE,partial_train_data_ABN ], partial_train_targets_ABN, validation_data=((val_data_BASELINE,val_data_ABNORMALITY), val_targets_ABNORMALITY), epochs=num_epochs, batch_size=32)
  History_Arr.append(history)
  Cross_model = get_model()
  Cross_model.compile(optimizer='RMSprop',loss='binary_crossentropy', metrics=['binary_accuracy'])
  plot()

In [0]:
def plot():
  acc = history.history['binary_accuracy']
  val_acc = history.history['val_binary_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()
  return
plot()

## FITTING COMPLETO

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

num_epochs = 40
history = DCCNN_MODEL.fit([baseline,abnormality ], abnormality_target_Categorical ,   epochs=num_epochs, batch_size=32)

## Evalutate the model

In [0]:
#evaluate the model

test_loss, test_acc = DCCNN_MODEL.evaluate((Test_baseline_Not_Shuffled,Test_abnormality_Not_Shuffled), Test_abnormality_target_Categorical,batch_size=32, verbose= 1)
print(test_loss)
print(test_acc)

## Load OR Save the model

In [0]:
#Save the model!
DCCNN_MODEL.save(os.path.join(base_dir,'Models/Model_Task_2.2_DCCN_Baseline_BaseVersion_0.86.h5'))

## Adding Data Augmentation 

### Data Augmentation

In [0]:
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))
test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))
test_images_Augmented = test_images_Augmented.astype('float32') / 65535


Test manipulation

In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 1


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,2)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,2)


In [0]:
print(Test_abnormality_Not_Shuffled)

In [0]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 1

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,2)
baseline_target_Categorical = to_categorical(baseline_target,2)



print(abnormality_target_Categorical[:10])

In [0]:
#ABNORMALITY OPERATIONS

print(abnormality_target)

In [0]:
print(base_train_data)
print(base_train_data.shape)

In [0]:
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def double_generator(train_base_images, train_abn_images, train_labels, subset, batch_size=32):

    gen = ImageDataGenerator(
      validation_split=0.1,
      rescale=1./65535,
      rotation_range=360,
      zoom_range=0.1, 
      horizontal_flip=True,
      vertical_flip = True,
      fill_mode='nearest'
    )

    gen.fit(train_abn_images)

    gen_abn = gen.flow(train_abn_images, train_labels,  batch_size=batch_size, subset=subset, seed=1)
    gen_base = gen.flow(train_base_images, train_labels, batch_size=batch_size, subset=subset, seed=1)

    while True:
        abn_img, abn_label = gen_abn.next()
        base_img, _ = gen_base.next()
        yield [base_img, abn_img], abn_label

train_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'training', batch_size=32)
validation_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'validation', batch_size=32)

### FITTING

In [0]:
history = DCCNN_MODEL.fit_generator(
      train_generator,
      steps_per_epoch=2408//32,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=268//32)

### Evalutate the model

In [0]:


test_loss, test_acc = DCCNN_MODEL.evaluate((Test_baseline_Not_Shuffled, Test_abnormality_Not_Shuffled),Test_abnormality_target_Categorical,batch_size=32,  verbose= 1)
print(test_loss)
print(test_acc)

### Load OR Save the model

In [0]:
#Save the model!
DCCNN_MODEL.save(os.path.join(base_dir,'Models/Model_Task_4_DCCNN_Binary_WithDataAugmentation_0.8869.h5'))

# USING dual-channel classification



## Model 

In [0]:
%tensorflow_version 1.14
!pip install keras==2.1.5
import tensorflow as tf
from tensorflow import keras
from keras.layers import merge, Input
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D

img_rows, img_cols = 150, 150
nb_filters_1 = 32
nb_filters_2 = 64
nb_filters_3 = 128
nb_filters_4 = 256
nb_conv = 3


init1 = keras.layers.Input(shape=(150,150,1),) #channel 1


init2 = keras.layers.Input(shape=(150,150,1),) #channel 2

fork11 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv), strides=(1,1), input_shape=(None, 150, 150, 1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(init1)
fork12 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv),strides=(1,1), input_shape=(None, 150, 150, 1), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(init2)
maxpool11 = keras.layers.MaxPooling2D(strides=(2,2),)(fork11)
maxpool22 = keras.layers.MaxPooling2D(strides=(2,2),)(fork12)

fork21 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool11)
fork22 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(maxpool22)

maxpool21 = keras.layers.MaxPooling2D(strides=(2,2), )(fork21)
maxpool22 = keras.layers.MaxPooling2D(strides=(2,2), )(fork22)
fork31 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation='relu',  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool21)
fork32 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation='relu',  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool22)

maxpool31 = keras.layers.MaxPooling2D(strides=(2,2), )(fork31)
maxpool32 = keras.layers.MaxPooling2D(strides=(2,2), )(fork32)
fork41 = keras.layers.Conv2D(nb_filters_4, (3, 3),strides=(1,1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool31)
fork42 = keras.layers.Conv2D(nb_filters_4, (3, 3), strides=(1,1), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(maxpool32)
merge4 = keras.layers.Subtract()([fork21, fork22, ]) #One single merge(Channel1, Channel 2)
flatten = keras.layers.Flatten()(merge4)
dense1 = keras.layers.Dense(256, activation="relu")(flatten)
dropout1 = keras.layers.Dropout(0.4)(dense1)
dense3 = keras.layers.Dense(128, activation="relu")(dropout1)
output = keras.layers.Dense(4, activation="softmax")(dense3)



DCCNN_MODEL = keras.models.Model((init1, init2), output)


In [0]:
DCCNN_MODEL.summary()

In [0]:
DCCNN_MODEL.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

## Data Manipulation

In [0]:
from tensorflow.keras.utils import to_categorical
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))


test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))

train_images_Augmented = train_images_Augmented.astype('float32') / 65535
test_images_Augmented = test_images_Augmented.astype('float32') / 65535

In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 2
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 3


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,4)


In [0]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 2
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 3

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,4)
baseline_target_Categorical = to_categorical(baseline_target,4)



print(abnormality[:10])

In [0]:

print(abnormality_target_Categorical)

## 10-Cross validation

In [0]:
#cross-validation, SUL TRAINING ESTRATTO e shuffled
import numpy as np
keras.layers.Conv2D?
keras.layers.MaxPooling2D?
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def get_model():

  img_rows, img_cols = 150, 150
  img_rows, img_cols = 150, 150
  nb_filters_1 = 32
  nb_filters_2 = 64
  nb_filters_3 = 128
  nb_filters_4 = 256
  nb_conv = 3


  init1 = keras.layers.Input(shape=(150,150,1),) #channel 1


  init2 = keras.layers.Input(shape=(150,150,1),) #channel 2

  fork11 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv), strides=(1,1), input_shape=(None, 150, 150, 1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(init1)
  fork12 = keras.layers.Conv2D(nb_filters_1, (nb_conv, nb_conv),strides=(1,1), input_shape=(None, 150, 150, 1), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(init2)
  maxpool11 = keras.layers.MaxPooling2D(strides=(2,2),)(fork11)
  maxpool22 = keras.layers.MaxPooling2D(strides=(2,2),)(fork12)

  fork21 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool11)
  fork22 = keras.layers.Conv2D(nb_filters_2, (nb_conv, nb_conv),strides=(1,1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(maxpool22)

  maxpool21 = keras.layers.MaxPooling2D(strides=(2,2), )(fork21)
  maxpool22 = keras.layers.MaxPooling2D(strides=(2,2), )(fork22)
  fork31 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation='relu',  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool21)
  fork32 = keras.layers.Conv2D(nb_filters_3, (nb_conv, nb_conv),strides=(1,1),  activation='relu',  kernel_regularizer=keras.regularizers.l2(0.001))(maxpool22)

  maxpool31 = keras.layers.MaxPooling2D(strides=(2,2), )(fork31)
  maxpool32 = keras.layers.MaxPooling2D(strides=(2,2), )(fork32)
  fork41 = keras.layers.Conv2D(nb_filters_4, (3, 3),strides=(1,1),  activation='relu', kernel_regularizer=keras.regularizers.l2(0.001) )(maxpool31)
  fork42 = keras.layers.Conv2D(nb_filters_4, (3, 3), strides=(1,1), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(maxpool32)
  merge4 = keras.layers.Subtract()([fork21, fork22, ]) #One single merge(Channel1, Channel 2)
  flatten = keras.layers.Flatten()(merge4)
  dense1 = keras.layers.Dense(256, activation="relu")(flatten)
  dropout1 = keras.layers.Dropout(0.4)(dense1)
  dense3 = keras.layers.Dense(128, activation="relu")(dropout1)
  output = keras.layers.Dense(4, activation="softmax")(dense3)


  Cross_model = keras.models.Model((init1, init2), output)
  return Cross_model

def plot():
  acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()
  return


############################# CROSS VALIDATION ######################

k = 10

num_epochs = 70
History_Arr =[]
Cross_model = get_model()
Cross_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

num_val_samples = len(baseline_Not_Shuffled) // k
for i in range(k):
  print('processing fold #', i)
  val_data_ABNORMALITY =  abnormality[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets_ABNORMALITY = abnormality_target_Categorical[i * num_val_samples: (i + 1) * num_val_samples]
  val_data_BASELINE = baseline[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data_ABN =    np.concatenate([abnormality[:i * num_val_samples],abnormality[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets_ABN = np.concatenate([abnormality_target_Categorical[:i * num_val_samples],abnormality_target_Categorical[(i + 1) * num_val_samples:]],axis=0)
  partial_train_data_BASE = np.concatenate([baseline[:i * num_val_samples],baseline[(i + 1) * num_val_samples:]],axis=0)
  history = Cross_model.fit([partial_train_data_BASE,partial_train_data_ABN ], partial_train_targets_ABN, validation_data=((val_data_BASELINE,val_data_ABNORMALITY), val_targets_ABNORMALITY), epochs=num_epochs, batch_size=32)
  History_Arr.append(history)
  Cross_model = get_model()
  Cross_model.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  plot()

## FITTING COMPLETO
Dopo la cross validation, fitto su tutto il DATASET

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

num_epochs = 60
history = DCCNN_MODEL.fit([baseline,abnormality ], abnormality_target_Categorical , callbacks=[callback],  epochs=num_epochs, batch_size=32)

## Evalutate the model

In [0]:
#evaluate the model

test_loss, test_acc = DCCNN_MODEL.evaluate((Test_baseline_Not_Shuffled,Test_abnormality_Not_Shuffled), Test_abnormality_target_Categorical,batch_size=32, verbose= 1)
print(test_loss)
print(test_acc)

## Load OR Save the model

In [0]:
#Save the model!
DCCNN_MODEL.save(os.path.join(base_dir,'Models/Model_Task_4_Dual-Channel_BaseVersion_0.52.h5'))

## Adding Data Augmentation 

### Data Augmentation

In [0]:
def load_training():
  train_images = np.load(os.path.join(base_dir,'Tensors/train_tensor.npy'))
  train_labels = np.load(os.path.join(base_dir,'Tensors/train_labels.npy'))
  test_images = np.load(os.path.join(base_dir,'Tensors/public_test_tensor.npy'))
  test_lables = np.load(os.path.join(base_dir,'Tensors/public_test_labels.npy'))
  return train_images,train_labels, test_images, test_lables
 
train_images_Augmented, train_labels_Augmented, test_images_Augmented, test_lable_Augmented = load_training()

train_images_Augmented = train_images_Augmented.reshape((5352, 150, 150,1))
test_images_Augmented = test_images_Augmented.reshape((672, 150, 150,1))
test_images_Augmented = test_images_Augmented.astype('float32') / 65535


Test manipulation

In [0]:
#TEST

from keras.utils import to_categorical
#splitting
Test_baseline_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_baseline_Not_Shuffled =  test_images_Augmented[::2]

Test_baseline_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_baseline_target_Not_Shuffled =  test_lable_Augmented[::2]

#ABNORMALITY OPERATIONS
Test_abnormality_Not_Shuffled =  np.empty_like(test_images_Augmented)
Test_abnormality_Not_Shuffled =   test_images_Augmented[1:][::2]

Test_abnormality_target_Not_Shuffled =  np.empty_like(test_lable_Augmented)
Test_abnormality_target_Not_Shuffled =   test_lable_Augmented[1:][::2] #ora contiene solo le label delle abnormality


#editing labels
leng = len(Test_abnormality_target_Not_Shuffled)
Test_abnormality_target_4 = np.empty_like(Test_abnormality_target_Not_Shuffled)
for i in range(leng):
    if Test_abnormality_target_Not_Shuffled[i] == 1:
      Test_abnormality_target_4[i] = 0
    if Test_abnormality_target_Not_Shuffled[i] == 2:
      Test_abnormality_target_4[i] = 1
    if Test_abnormality_target_Not_Shuffled[i] == 3:
      Test_abnormality_target_4[i] = 2
    if Test_abnormality_target_Not_Shuffled[i] == 4:
      Test_abnormality_target_4[i] = 3


#TO_CATEGORICAL
Test_abnormality_target_Categorical = to_categorical(Test_abnormality_target_4,4)
Test_baseline_target_Categorical = to_categorical(Test_baseline_target_Not_Shuffled,4)


In [0]:
print(Test_abnormality_Not_Shuffled)

In [0]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


#splitting
baseline_Not_Shuffled =  np.empty_like(train_images_Augmented)
baseline_Not_Shuffled =  train_images_Augmented[::2]

baseline_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
baseline_target_Not_Shuffled =  train_labels_Augmented[::2]

#ABNORMALITY OPERATIONS
abnormality_Not_Shuffled =  np.empty_like(train_images_Augmented)
abnormality_Not_Shuffled =   train_images_Augmented[1:][::2]

abnormality_target_Not_Shuffled =  np.empty_like(train_labels_Augmented)
abnormality_target_Not_Shuffled =   train_labels_Augmented[1:][::2] #ora contiene solo le label delle abnormaluity

#shuffling

abnormality, abnormality_target = shuffle( abnormality_Not_Shuffled, abnormality_target_Not_Shuffled, random_state=42)
baseline, baseline_target = shuffle( baseline_Not_Shuffled, baseline_target_Not_Shuffled, random_state=42)

#editing labels
leng = len(abnormality_target)
abnormality_target_4 = np.empty_like(abnormality_target)
for i in range(leng):
    if abnormality_target[i] == 1:
      abnormality_target_4[i] = 0
    if abnormality_target[i] == 2:
      abnormality_target_4[i] = 1
    if abnormality_target[i] == 3:
      abnormality_target_4[i] = 2
    if abnormality_target[i] == 4:
      abnormality_target_4[i] = 3

#categorical

abnormality_target_Categorical = to_categorical(abnormality_target_4,4)
baseline_target_Categorical = to_categorical(baseline_target,4)



print(abnormality_target_Categorical[:10])

In [0]:
#ABNORMALITY OPERATIONS

print(abnormality_target)

In [0]:
print(base_train_data)
print(base_train_data.shape)

In [0]:
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def double_generator(train_base_images, train_abn_images, train_labels, subset, batch_size=32):

    gen = ImageDataGenerator(
      validation_split=0.1,
      rescale=1./65535,
      rotation_range=360,
      zoom_range=0.1, 
      horizontal_flip=True,
      vertical_flip = True,
      fill_mode='nearest'
    )

    gen.fit(train_abn_images)

    gen_abn = gen.flow(train_abn_images, train_labels,  batch_size=batch_size, subset=subset, seed=1)
    gen_base = gen.flow(train_base_images, train_labels, batch_size=batch_size, subset=subset, seed=1)

    while True:
        abn_img, abn_label = gen_abn.next()
        base_img, _ = gen_base.next()
        yield [base_img, abn_img], abn_label

train_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'training', batch_size=32)
validation_generator = double_generator(baseline, abnormality, abnormality_target_Categorical, 'validation', batch_size=32)

### FITTING

In [0]:
history = DCCNN_MODEL.fit_generator(
      train_generator,
      steps_per_epoch=2408//32,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=268//32)

### Evalutate the model

In [0]:


test_loss, test_acc = DCCNN_MODEL.evaluate((Test_baseline_Not_Shuffled, Test_abnormality_Not_Shuffled),Test_abnormality_target_Categorical,batch_size=32,  verbose= 1)
print(test_loss)
print(test_acc)

### Load OR Save the model

In [0]:
#Save the model!
DCCNN_MODEL.save(os.path.join(base_dir,'Models/Model_Task_4_DualChannel_Corretto_WithDataAugmentation_0.5982.h5'))